In [46]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.summarizers.luhn import LuhnSummarizer
from sumy.summarizers import AbstractSummarizer
from sumy.summarizers.kl import KLSummarizer
from sumy.summarizers.lsa import LsaSummarizer
from transformers import pipeline,T5ForConditionalGeneration, T5Tokenizer
from summarizer.sbert import SBertSummarizer
from summarizer import Summarizer
from nltk.corpus import stopwords 
import wikipedia
import en_core_web_sm

In [50]:
userInput = input("Enter the text you are searching for in MiniWiki : ")
userNumber = int(input("Enter number of sentences you want : "))
try:
    wikisearch = wikipedia.page("m"+userInput)
except wikipedia.exceptions.DisambiguationError as e:
    print("Error: {0}".format(e))

wikicontent = wikisearch.content[:2500]
parser=PlaintextParser.from_string(wikicontent,Tokenizer("english"))

Enter the text you are searching for in MiniWikiMusic
Enter number of sentences you want: 4


In [51]:
lst1 = []
summarizer = LexRankSummarizer()
summary = summarizer(parser.document,userNumber)
for sentence in summary:
    print(sentence)


['Music is the art of arranging sounds in time through the elements of melody, harmony, rhythm, and timbre.',
 'Music is performed with a vast range of instruments and vocal techniques ranging from singing to rapping; there are solely instrumental pieces, solely vocal pieces (such as songs without instrumental accompaniment) and pieces that combine singing and instruments.',
 'Common sayings such as "the harmony of the spheres" and "it is music to my ears" point to the notion that music is often ordered and pleasant to listen to.',
 'Indeed, throughout history, some new forms or styles of music have been criticized as "not being music", including Beethoven\'s Grosse Fuge string quartet in 1825, early jazz in the beginning of the 1900s and hardcore punk in the 1980s.']

In [52]:
lst2 = []
summarizer = LuhnSummarizer()
summary = summarizer(parser.document,userNumber)
for sentence in summary:
    print(sentence)
lst2

['Music is performed with a vast range of instruments and vocal techniques ranging from singing to rapping; there are solely instrumental pieces, solely vocal pieces (such as songs without instrumental accompaniment) and pieces that combine singing and instruments.',
 'The word derives from Greek μουσική (mousiké; "(art) of the Muses").In its most general form, the activities describing music as an art form or cultural activity include the creation of works of music (songs, tunes, symphonies, and so on), the criticism of music, the study of the history of music, and the aesthetic examination of music.',
 'Common sayings such as "the harmony of the spheres" and "it is music to my ears" point to the notion that music is often ordered and pleasant to listen to.',
 'Music ranges from strictly organized compositions—such as Classical music symphonies from the 1700s and 1800s—through to spontaneously played improvisational music such as jazz, and avant-garde styles of chance-based contempora

In [53]:
lst3 = []
summarizer = KLSummarizer()
summary = summarizer(parser.document,userNumber)
for sentence in summary:
    print(sentence)
    
lst3

['The word derives from Greek μουσική (mousiké; "(art) of the Muses").In its most general form, the activities describing music as an art form or cultural activity include the creation of works of music (songs, tunes, symphonies, and so on), the criticism of music, the study of the history of music, and the aesthetic examination of music.',
 '"The creation, performance, significance, and even the definition of music vary according to culture and social context.',
 'There are many types of music, including popular music, traditional music, art music, music written for religious ceremonies, and work songs such as chanteys.',
 'Music ranges from strictly organized compositions—such as Classical music symphonies from the 1700s and 1800s—through to spontaneously played improvisational music such as jazz, and avant-garde styles of chance-based contemporary music from the 20th and 21st centuries.']

In [54]:
lst4 = []
summarizer = pipeline("summarization")
summarized = summarizer(wikicontent, min_length=75, max_length=100)
print(summarized[0]['summary_text'])
lst4

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 (https://huggingface.co/sshleifer/distilbart-cnn-12-6)


[' Music is the art of arranging sounds in time through the elements of melody, harmony, rhythm, and timbre . Different styles or types of music may emphasize, de-emphasize or omit some of these elements . Music is performed with a vast range of instruments and vocal techniques ranging from singing to rapping . Ancient Greek and Indian philosophers defined music in two parts: melodies, as melodies, and harmonies as tones ordered vertically .']

In [55]:
lst5 = []
summarizer_2 = LsaSummarizer()
summary_2 =summarizer_2(parser.document,userNumber)
for sentence in summary_2:
    print(sentence)
lst5

['Music is performed with a vast range of instruments and vocal techniques ranging from singing to rapping; there are solely instrumental pieces, solely vocal pieces (such as songs without instrumental accompaniment) and pieces that combine singing and instruments.',
 'Ancient Greek and Indian philosophers defined music in two parts: melodies, as tones ordered horizontally, and harmonies as tones ordered vertically.',
 'However, 20th-century composer John Cage thought that any sound can be music, saying, for example, "There is no noise, only sound.',
 'Indeed, throughout history, some new forms or styles of music have been criticized as "not being music", including Beethoven\'s Grosse Fuge string quartet in 1825, early jazz in the beginning of the 1900s and hardcore punk in the 1980s.']

In [56]:
model = T5ForConditionalGeneration.from_pretrained("t5-base")
tokenizer = T5Tokenizer.from_pretrained("t5-base")


In [57]:
inputs = tokenizer.encode("summarize: " + wikicontent, return_tensors="pt", max_length=512, truncation=True)

In [58]:
lst6 = []
outputs = model.generate(
    inputs, 
    max_length=150, 
    min_length=40, 
    length_penalty=2.0, 
    num_beams=4, 
    early_stopping=True)
print(tokenizer.decode(outputs[0]).replace("<pad>","").replace("</s>",""))
lst6

[' music is the art of arranging sounds in time through the elements of melody, harmony, rhythm, and timbre. ancient Greek and Indian philosophers defined music in two parts: melodies, as tones ordered horizontally, and harmonies as tones ordered vertically.']

In [59]:
body = wikicontent
model = SBertSummarizer('paraphrase-MiniLM-L6-v2')
result = model(body, num_sentences=userNumber)
result

'Music is the art of arranging sounds in time through the elements of melody, harmony, rhythm, and timbre. It is one of the universal cultural aspects of all human societies. General definitions of music include common elements such as pitch (which governs melody and harmony), rhythm (and its associated concepts tempo, meter, and articulation), dynamics (loudness and softness), and the sonic qualities of timbre and texture (which are sometimes termed the "color" of a musical sound). Music is performed with a vast range of instruments and vocal techniques ranging from singing to rapping; there are solely instrumental pieces, solely vocal pieces (such as songs without instrumental accompaniment) and pieces that combine singing and instruments.'